In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
import re
import time
from datetime import datetime
import os
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import paramiko


user_name='devansh/'

ssh_host = '10.120.1.50'
ssh_username = 'research-user'
localhost = '127.0.0.1'

# prod
ezcapc_user = 'read_only'
ezcapc_pwd = 'WHC63fKyjudfSz2U'
ezcapc_host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ezcapc_db_name = 'CAPC_APIGATEWAY'
env='production'
paas_user = 'read_only'
paas_pwd = 'eUN2cORh1sCRG998'
paas_host = 'paas-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
paas_db_name = 'FACILITY_SERVICE'

# # stag
# ezcapc_user = 'read_only'
# ezcapc_pwd = '3j4A3PaQc8BV050R'
# ezcapc_host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# ezcapc_db_name = 'CAPC_APIGATEWAY'
# env='staging'
# paas_user = 'read_only'
# paas_pwd = 's9a843QvgkPje5SD'
# paas_host = 'paas-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# paas_db_name = 'METADATA_SERVICE'



# ezcapc_db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(ezcapc_user,ezcapc_pwd,ezcapc_host,ezcapc_db_name)
# ezcapc_db_connection = create_engine(ezcapc_db_connection_str)

# paas_db_connection_str = 'mysql+pymysql://{}:{}@{}'.format(paas_user,paas_pwd,paas_host)
# paas_db_connection = create_engine(paas_db_connection_str)

In [5]:
# capc_sql_hostname = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# sql_username = 'read_only'
# capc_sql_password = 'WHC63fKyjudfSz2U'


# def get_data_by_db_connection(sql_hostname, sql_password, sql_db, query):
#     with SSHTunnelForwarder(
#         (ssh_host, ssh_port),
#         ssh_username=ssh_user,
#         ssh_pkey='/home/devanshparmar/pem/research_user.pem',
#         remote_bind_address=(sql_hostname, 3306)) as tunnel:
#         db_con = pymysql.connect(host= sql_ip, user="read_only",
#             passwd=sql_password,
#             database=sql_db,
#             port=tunnel.local_bind_port)
#         print(tunnel.local_bind_port)
#         data = pd.read_sql_query(query, db_con)
#         db_con.close()
#     return data

# ssh_host = '10.120.1.50'
# ssh_user = 'research-user'
# ssh_port = 22
# sql_ip = '127.0.0.1'

# display(get_data_by_db_connection(ezcapc_host, ezcapc_pwd, ezcapc_db_name,'select * from account_mst order by created_date desc limit 1'))

In [2]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global ezcapc_tunnel
    global paas_tunnel
    ezcapc_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/devanshparmar/pem/research_user.pem',
        remote_bind_address = (ezcapc_host, 3306)
    )
    paas_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/devanshparmar/pem/research_user.pem',
        remote_bind_address = (paas_host, 3306)
    )
    ezcapc_tunnel.start()
    paas_tunnel.start()
    print(ezcapc_tunnel.is_active)
    print(paas_tunnel.is_active)

In [3]:
def mysql_connect():
    global ezcapc_connection
    global paas_connection
    ezcapc_connection = pymysql.connect(
        host=localhost,
        user=ezcapc_user,
        passwd=ezcapc_pwd,
        database=ezcapc_db_name,
        port=ezcapc_tunnel.local_bind_port
    )

    paas_connection = pymysql.connect(
        host=localhost,
        user=paas_user,
        passwd=paas_pwd,
        database=paas_db_name,
        port=paas_tunnel.local_bind_port
    )

In [32]:
open_ssh_tunnel()
mysql_connect()

True
True


In [5]:
# type_id=[1491521,1491533,1491537,1491525,1491532]

In [7]:
acc=[]
# for i in open('/home/devanshparmar/devansh/Download/accounts/sjrmc_july','r').readlines():
#     acc.append(i.rstrip('\n'))

# acc=pd.read_csv('/home/devanshparmar/Documents/python scripts/HMCA_demographic.csv', sep='\t', index_col=False).encounter_id.to_list()

# fac=[45,49,68,62,56,82,77,73,72,69,67,66
# ]

In [6]:
# acc

In [27]:
paas_facility=pd.read_sql('''
SELECT 
    id
FROM
    facility_master
WHERE
    client_id IN (15, 13)
    ''',paas_connection)

In [28]:
facilities=paas_facility.id.to_list()

In [29]:
facilities

[16,
 21,
 22,
 23,
 28,
 29,
 31,
 32,
 35,
 37,
 38,
 39,
 40,
 201,
 202,
 203,
 204,
 205,
 219,
 220,
 221,
 245,
 251,
 253,
 256,
 257,
 258,
 332,
 333,
 338,
 339,
 379,
 380,
 389,
 20,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 1

In [34]:
df_all=[]

for f_id in facilities:
    query ='''
    SELECT DISTINCT
        am.account_number,
        dpd.nlp_request_s3_path,
        dpd.nlp_ner_entities_result_s3_path,
        am.facility_id,
        dm.service_date,
        dpd.nlp_pcs_xml_result_s3_path,
        dpd.preprocessing_request_s3_path,
        dpd.preprocessing_result_s3_path,
        dm.orignal_path,
        dm.type_id,
        dpd.created_date,
        dpd.ocr_request_s3_path,
        dpd.ocr_result_s3_path,
        dpd.eandmcs_request_s3_path,
        dpd.eandmcs_result_s3_path,
        dpd.ruleengine_request_s3_path,
        dpd.cptcs_request_s3_path,
        dpd.cptcs_result_s3_path,
        dm.account_id,
        dpd.id AS DPD_ID,
        dm.path AS filePath,
        dm.id AS document_id,
        em.id AS encounter_id,
        dpd.ocr_request_id
    FROM
        account_mst am
            INNER JOIN
        document_mst dm ON dm.account_id = am.id
            AND dm.is_active = 1
            INNER JOIN
        encounter_mst em ON dm.account_id = em.account_id
            AND dm.service_date = em.service_date
            AND dm.physician_id = em.physician_id
            AND em.is_active = 1
            LEFT JOIN
        document_processing_detail dpd ON dpd.document_id = dm.id
            
    WHERE
        dm.is_physician_document = 1
            AND am.is_active = 1
            and am.facility_id in ({})
            and dpd.created_date > '2023-07-15'
    AND dpd.id = (SELECT 
                MAX(id)
            FROM
                document_processing_detail
            WHERE
                document_id = dm.id)
    ORDER BY dm.created_date DESC limit 20;
    '''.format(f_id)
    # '''.format(','.join('"'+str(ac)+'"' for ac in acc))

    # print(query)
    df_all.append(pd.read_sql(query, con=ezcapc_connection))
    print(len(df_all))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [35]:
df_query=pd.concat(df_all)

In [36]:
# df_query[df_query.account_number=='550001701957'].nlp_ner_entities_result_s3_path.to_list()

In [37]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [38]:
# print(df_query[['ocr_request_id','account_number']].to_markdown())

In [39]:
# query_type_id = '''select id as type_id, code as type_code, description as type_desc from METADATA_SERVICE.facility_metadata 
# where facility_id in ({})'''.format(','.join( str(i) for i in df_query.facility_id.unique().tolist()))


# df_type_id=pd.read_sql(query_type_id, con=paas_connection)

In [40]:
# df_query = pd.merge(df_query, df_type_id, on='type_id' , how='inner')

In [41]:
df_query=df_query.groupby(["account_number","service_date", 'facility_id', 'type_id']).first().reset_index()

In [42]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [43]:
# df_query = df_query.groupby('facility_id').head(30).reset_index(drop=True)

In [44]:
# df_query

In [52]:
df_query.groupby('facility_id', group_keys=False).apply(lambda x : do_all(x))

# do_all(df_query)

[16]
4
[20]
3
[21]
[22]
[23]
2
[28]
2
[29]
[31]
2
[32]
4
[35]
[37]
2
[38]
2
[39]
10
[40]
2
[41]
7
[46]
1
[47]
4
[48]
1
[49]
1
[50]
1
[51]
1
[52]
2
[53]
1
[54]
4
[55]
2
[56]
2
[57]
1
[58]
2
[59]
1
[60]
1
[62]
3
[63]
4
[64]
2
[66]
4
[67]
2
[68]
1
[69]
2
[70]
1
[71]
2
[72]
2
[73]
4
[74]
1
[75]
4
[76]
4
[77]
1
[78]
2
[79]
1
[80]
6
[81]
6
[82]
5
[83]
5
[84]
2
[85]
2
[88]
3
[93]
2
[94]
1
[95]
1
[96]
2
[97]
1
[98]
2
[99]
1
[100]
4
[101]
1
[102]
4
[103]
1
[104]
2
[105]
1
[106]
3
[107]
6
[108]
8
[109]
1
[110]
2
[111]
5
[112]
3
[113]
2
[114]
4
[116]
2
[117]
5
[118]
9
[119]
5
[120]
4
[121]
8
[122]
9
[123]
8
[124]
4
[125]
1
[127]
6
[128]
4
[130]
2
[131]
8
[132]
2
[133]
6
[134]
4
[135]
2
[136]
6
[137]
1
[139]
7
[140]
5
[141]
2
[142]
2
[143]
5
[144]
1
[145]
1
[148]
6
[150]
5
[151]
4
[152]
3
[153]
7
[154]
1
[156]
2
[158]
4
[159]
1
[160]
5
[161]
4
[162]
3
[165]
3
[166]
3
[167]
9
[172]
1
[173]
5
[174]
9
[175]
5
[176]
2
[177]
5
[178]
11
[179]
2
[180]
1
[183]
1
[184]
4
[185]
2
[186]
2
[187]
3
[188]
6
[19

""


In [121]:
# df_query.drop_duplicates()

In [16]:
# print(df_query.groupby('facility_id').size().to_markdown())

In [59]:
df_query[df_query.ocr_request_id=='6234828']

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,DPD_ID,filePath,document_id,encounter_id,ocr_request_id


In [51]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                              df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             # df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            #  df_query.type_code
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_pcs.xml".format(env, pcs, fac_name,enc_id, acc, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_request.json".format(env, em_req,fac_name, enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        # cmd.append((em_req, '{}/{}/{}_{}_eandm_req.json'.format(fac_name,dpd_id, enc_id, acc)))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        # cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))
        
        # cmd.append((pre_req, '{}/{}/{}_{}_pre_req.txt'.format(fac_name,dpd_id, enc_id,acc)))
        # cmd.append((pre_res, '{}/{}/{}_{}_pre_res.json'.format(fac_name,dpd_id, enc_id,acc)))

        # temp HMCA
        # cmd.append((pre_req, '{}/{}/{}_{}_pre_req.txt'.format(fac_name, tc, enc_id,acc)))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.pdf".format(env,ori_path,fac_name,acc_id,acc))
        # cmd.append(('ezcapc/'+ori_path, '{}/{}/{}_{}_doc.pdf'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))
        
        # original files
        if ori_path != None:
            cmd.append(('ezcapc/'+ori_path, '{}/{}/{}_{}_{}_pdf.pdf'.format(fac_name,dpd_id, enc_id,acc, t_id)))
        # if file_path != None:
        #     cmd.append(('ezcapc/'+file_path, '{}/{}/{}_{}.txt'.format(fac_name,t_id, acc, file_path)))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, enc_id, acc)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, enc_id, acc)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)
    
    if cmd==[]:
        return None, None, None
    
    cmd.sort()
    commands=[]
    
    # for c in cmd:
    #     temp=c[0].rsplit('/', 1)
    #     if len(commands) == 0:
    #         commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
    #     elif temp[0] in commands[-1]:
    #         if 'exclude' in commands[-1]:
    #             commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
    #     else:
    #         commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    # return cmd, commands, fac_name

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [47]:
# def print_downlaod_files(commands,cmd, fac):
#     print(len(commands))
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     # file=open("/home/devanshparmar/devansh/Download/{}_docs_download_{}.sh".format(fac,time_now),'w')
#     file=open("/home/devanshparmar/devansh/Download/HMCA_preprocess.sh",'a')
    
#     for f in commands:
#         file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + f +'\n')
#         # file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + f +'\n')
#     # file.write('sh {}_name_change_*'.format(fac))
    
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}_{}.tar.xz {}\n'.format(fac, time_now,fac))
#     file.write('rm -r {}\n'.format(fac))
#     file.close()


def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    # file=open("/home/devanshparmar/devansh/Download/{}_docs_download_{}.sh".format(fac,time_now),'w')
    file=open("/home/devanshparmar/devansh/Download/scp_ph_pdf.sh",'a')
    
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + f +'\n')
        # file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + f +'\n')
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" ' + f[1]+'\n')
    file.write('tar -cJf docs_{}_{}.tar.xz {}\n'.format(fac, time_now,fac))
    file.write('rm -r {}\n'.format(fac))
    file.close()

In [48]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [49]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    if cmd==None or commands==None or fac==None:
        return
    
    if cmd==[] or commands==[] or fac=='':
        return
    
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [18]:
df_query.columns

Index(['account_number', 'nlp_request_s3_path',
       'nlp_ner_entities_result_s3_path', 'facility_id', 'service_date',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'type_id',
       'created_date', 'ocr_request_s3_path', 'ocr_result_s3_path',
       'eandmcs_request_s3_path', 'eandmcs_result_s3_path',
       'ruleengine_request_s3_path', 'cptcs_request_s3_path',
       'cptcs_result_s3_path', 'account_id', 'DPD_ID', 'filePath',
       'document_id', 'encounter_id'],
      dtype='object')

In [21]:
# df_query[df_query.account_number.str.contains('0996')]

In [22]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()

In [26]:
df_query[df_query.facility_id==38]

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_request_s3_path,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,DPD_ID,filePath,document_id,encounter_id
